When we have to add Sy

In [1]:
function Hamiltonian2(N,B)
    
    #Dimension of the Hilbert Space
    dim = (2)^N
    #Initialize a matrix for the Hamiltonian
    H = zeros(dim,dim)
    
    for ket in (0:dim-1)
        
        #Put ket in binary for printing purposes
        ket_binary = bitstring(ket)
        #Initialize the matrix element we compute
        Diagonal = Int64(0)
        
        for SpinIndex in (0:N-1) 
            bit = Int(2)^(SpinIndex)
            bra = ket ⊻ bit
            #can use the same +/- 1 mechanism that we use in the diagonal piece
            #not sure if this is exactly right though
            H[bra+1,ket+1] += (2*((ket>>SpinIndex)&1)-1)*B

            if SpinIndex == N-1
                break
            end
            
            Si = 2*((ket>>SpinIndex)&1)-1
            Si_next = 2*((ket>>(SpinIndex+1))&1)-1
            Bond = Si*Si_next
            Diagonal += Bond
        end
        H[ket+1,ket+1] += -1 * Diagonal
    end
    return H
end

Hamiltonian2 (generic function with 1 method)